In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
result_folder = os.path.join("..", "experiment_cmpb", "hippocampus",
                             "fold", 'image_results')
exps = ['cn_ad', 'cn_mci', 'mci_ad']
fts = ['gm', 'wm', 'csf', 'tissues']
folds = range(10)

In [3]:
class ColumnExtractor(object):
    def __init__(self, str_):
        self.str_ = str_

    def transform(self, X):
        return X.iloc[:, X.columns.str.contains(self.str_)]

    def fit(self, X, y=None):
        return self

In [4]:
results = []
for exp in exps:
    for fold in folds:
        path = os.path.join(result_folder.replace("fold", str(fold)), exp, "clf.pickle")

        infile = open(path, 'rb')
        new_dict = pickle.load(infile)
        infile.close()
        for j in range(len(new_dict)):
            res = new_dict[j]['scores']
            res['exp'] = exp
            res['fold'] = fold
            res['ft'] = new_dict[j]['fts']
            
            results.append(res)
results = pd.DataFrame(results)
results.head(10)

,train,f1,acc,bacc,sens,spe,auc,exp,fold,ft
0,0.945294,0.904762,0.923077,0.920123,0.904762,0.935484,0.963134,cn_ad,0,gm
1,0.904602,0.782609,0.807692,0.815668,0.857143,0.774194,0.937020,cn_ad,0,wm
2,0.959781,0.857143,0.884615,0.880184,0.857143,0.903226,0.958525,cn_ad,0,csf
3,0.968706,0.857143,0.884615,0.880184,0.857143,0.903226,0.975422,cn_ad,0,tissues
4,0.954671,0.772727,0.803922,0.804762,0.809524,0.800000,0.915873,cn_ad,1,gm
5,0.908500,0.780488,0.823529,0.814286,0.761905,0.866667,0.898413,cn_ad,1,wm
6,0.958598,0.818182,0.843137,0.845238,0.857143,0.833333,0.952381,cn_ad,1,csf
7,0.973023,0.850000,0.882353,0.871429,0.809524,0.933333,0.946032,cn_ad,1,tissues
8,0.938757,0.878049,0.901961,0.895238,0.857143,0.933333,0.969841,cn_ad,2,gm
9,0.895245,0.829268,0.862745,0.854762,0.809524,0.900000,0.934921,cn_ad,2,wm


In [5]:
def print_mean_results(dt, exp, round=3):
    cols = ['auc', 'acc', 'sens', 'spe', 'f1', 'ft']
    dt = dt[dt['exp']==exp][cols]
    means = (dt.groupby(['ft']).mean()*100).round(round)
    stds = (dt.groupby(['ft']).std(ddof=0)*100).round(round)
    metrics = means.columns.values
    fts = means.index.values
    results = means.copy()
    for metric in metrics:
        for ft in fts:
            if metric in ['auc', 'f1']:
                means.loc[ft, metric] = (means.loc[ft, metric]/100).round(round)
                stds.loc[ft, metric] = (stds.loc[ft, metric]/100).round(round)
            results.loc[ft, metric] = str(means.loc[ft, metric]) + \
                                      ' +- ' + str(stds.loc[ft, metric])
    return results

In [6]:
def get_latex_table(dt, exp, round=3):

    fts_name = {'gm':'GM', 
                'wm': 'WM',
                'csf': 'CSF',
                'tissues': 'GM\&WM\&CSF'}
    
    cols = ['auc', 'acc', 'sens', 'spe', 'f1', 'ft']
    dt = dt[dt['exp']==exp][cols]
    means = (dt.groupby(['ft']).mean()*100).round(round)
    stds = (dt.groupby(['ft']).std(ddof=0)*100).round(round)
    metrics = means.columns.values
    fts = means.index.values
    
    for ft in fts:
        text = "\\textbf{"+fts_name[ft]+"}"
        for metric in metrics:
            if metric in ['auc', 'f1']:
                means.loc[ft, metric] = (means.loc[ft, metric]/100).round(round)
                stds.loc[ft, metric] = (stds.loc[ft, metric]/100).round(round)

            text += ' & $' + str(means.loc[ft, metric]).replace(".", ',') + \
                    "\pm" + str(stds.loc[ft, metric]).replace(".", ',')+"$ "
        text += "\\tabularnewline"
        print(text)
        print("\midrule")

In [7]:
for exp in exps:
    print(exp)
    display(print_mean_results(results, exp, round=2))

cn_ad


,auc,acc,sens,spe,f1
ft,,,,,
csf,0.94 +- 0.04,86.3 +- 4.12,83.67 +- 7.97,88.08 +- 6.0,0.83 +- 0.05
gm,0.94 +- 0.03,87.47 +- 3.96,85.1 +- 6.52,89.04 +- 6.0,0.85 +- 0.05
tissues,0.95 +- 0.03,89.24 +- 4.04,85.57 +- 8.5,91.72 +- 5.0,0.87 +- 0.05
wm,0.9 +- 0.03,80.82 +- 4.19,80.86 +- 3.7,80.78 +- 8.54,0.78 +- 0.03


cn_mci


,auc,acc,sens,spe,f1
ft,,,,,
csf,0.82 +- 0.04,74.87 +- 3.44,74.14 +- 7.03,75.51 +- 6.12,0.73 +- 0.04
gm,0.8 +- 0.06,74.69 +- 5.3,70.57 +- 9.76,78.12 +- 8.36,0.72 +- 0.06
tissues,0.83 +- 0.05,75.58 +- 3.57,72.94 +- 8.41,77.81 +- 6.84,0.73 +- 0.04
wm,0.77 +- 0.04,70.35 +- 4.52,69.69 +- 9.22,70.87 +- 7.66,0.68 +- 0.06


mci_ad


,auc,acc,sens,spe,f1
ft,,,,,
csf,0.72 +- 0.07,66.96 +- 8.34,66.14 +- 10.45,67.9 +- 8.69,0.68 +- 0.09
gm,0.66 +- 0.15,66.75 +- 6.03,73.74 +- 12.95,58.36 +- 17.47,0.7 +- 0.06
tissues,0.73 +- 0.07,69.8 +- 6.73,74.09 +- 6.3,64.57 +- 11.17,0.73 +- 0.06
wm,0.7 +- 0.09,67.64 +- 8.08,74.2 +- 11.75,59.79 +- 10.12,0.71 +- 0.08


In [8]:
for exp in exps:
    print(exp)
    display(get_latex_table(results, exp, round=2))

cn_ad
\textbf{CSF} & $0,94\pm0,04$  & $86,3\pm4,12$  & $83,67\pm7,97$  & $88,08\pm6,0$  & $0,83\pm0,05$ \tabularnewline
\midrule
\textbf{GM} & $0,94\pm0,03$  & $87,47\pm3,96$  & $85,1\pm6,52$  & $89,04\pm6,0$  & $0,85\pm0,05$ \tabularnewline
\midrule
\textbf{GM\&WM\&CSF} & $0,95\pm0,03$  & $89,24\pm4,04$  & $85,57\pm8,5$  & $91,72\pm5,0$  & $0,87\pm0,05$ \tabularnewline
\midrule
\textbf{WM} & $0,9\pm0,03$  & $80,82\pm4,19$  & $80,86\pm3,7$  & $80,78\pm8,54$  & $0,78\pm0,03$ \tabularnewline
\midrule


None

cn_mci
\textbf{CSF} & $0,82\pm0,04$  & $74,87\pm3,44$  & $74,14\pm7,03$  & $75,51\pm6,12$  & $0,73\pm0,04$ \tabularnewline
\midrule
\textbf{GM} & $0,8\pm0,06$  & $74,69\pm5,3$  & $70,57\pm9,76$  & $78,12\pm8,36$  & $0,72\pm0,06$ \tabularnewline
\midrule
\textbf{GM\&WM\&CSF} & $0,83\pm0,05$  & $75,58\pm3,57$  & $72,94\pm8,41$  & $77,81\pm6,84$  & $0,73\pm0,04$ \tabularnewline
\midrule
\textbf{WM} & $0,77\pm0,04$  & $70,35\pm4,52$  & $69,69\pm9,22$  & $70,87\pm7,66$  & $0,68\pm0,06$ \tabularnewline
\midrule


None

mci_ad
\textbf{CSF} & $0,72\pm0,07$  & $66,96\pm8,34$  & $66,14\pm10,45$  & $67,9\pm8,69$  & $0,68\pm0,09$ \tabularnewline
\midrule
\textbf{GM} & $0,66\pm0,15$  & $66,75\pm6,03$  & $73,74\pm12,95$  & $58,36\pm17,47$  & $0,7\pm0,06$ \tabularnewline
\midrule
\textbf{GM\&WM\&CSF} & $0,73\pm0,07$  & $69,8\pm6,73$  & $74,09\pm6,3$  & $64,57\pm11,17$  & $0,73\pm0,06$ \tabularnewline
\midrule
\textbf{WM} & $0,7\pm0,09$  & $67,64\pm8,08$  & $74,2\pm11,75$  & $59,79\pm10,12$  & $0,71\pm0,08$ \tabularnewline
\midrule


None